## Example IMC analysis with Morpheus

#### Step 0: import required packages

In [1]:
import morpheus as mp
from morpheus.classification import train, PatchClassifier
from morpheus.counterfactual import generate_cf
from lightning.pytorch import seed_everything

# Set seed for reproducibility
seed_everything(42)

Seed set to 42


42

#### Step 1: create a dataset object

In [2]:
data_path = "/groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/crc.h5"  # change to your own directory
livertumor = mp.SpatialDataset(data_path=data_path)

#### Step 2: generate data splits to prepare for model training

Next, we will need to generate train, validation, and test data splits for model training. We want to stratify our splits by the label we want to predict.

In [3]:
labelname = "Tcytotoxic"
livertumor.generate_data_splits(stratify_by=labelname)

Data splits already exist in /groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/data


##### Step 3: train PyTorch model

In [ ]:
# initialize model
model_arch = "unet"
n_channels = len(livertumor.channel_names)
img_size = livertumor.data_dim[1:3]
model = PatchClassifier(n_channels, img_size, model_arch=model_arch)

# train model
model = train(model=model, dataset=livertumor, labelname=labelname)

#### Step 4: generate counterfactuals

In [ ]:
# Usage example
generate_cf_params = {
    "model_path": "path/to/your/model",
    "channel_to_perturb": ["CXCR4", "PD1", "PD-L1"],
    "data_dict": {...},
}

# Generate counterfactuals
cf = generate_cf(livertumor, model, generate_cf_params)

In [ ]:
metadata = livertumor.metadata[["PatientID", "ImageNumber"]]
label = livertumor.label

# merge metadata and label using ImageNumber as the common column
metadata = metadata.merge(label, on="ImageNumber")
metadata = metadata.reset_index().rename(columns={"index": "patch_index"})

# add misc to metadata with columns "location_x_index" and "location_y_index"
metadata = metadata.join(
    pd.DataFrame(misc, columns=["location_x_index", "location_y_index"])
)

metadata.to_csv(
    "/groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/metadata.csv",
    index=False,
)

In [ ]:
import h5py

with h5py.File(
    "/groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/crc.h5", "w"
) as f:
    # Create a dataset to store the images
    dset = f.create_dataset(
        "images",
        data=livertumor.intensity,
        compression="gzip",
        chunks=(100, 16, 16, 44),
        dtype=livertumor.intensity.dtype,
    )

    # Create a group to store the metadata
    metadata_numpy = livertumor.metadata.to_records(index=False)
    dset_metadata = f.create_dataset(
        "metadata", data=metadata_numpy, dtype=metadata_numpy.dtype
    )

    # Create a group to store the channel names
    dset_channel_names = f.create_dataset("channel_names", data=livertumor.channel)